In [1]:
!pip install config
!pip install apscheduler
!pip install oandapyV20
!pip install oanda-candles

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [ ]:
import pandas as pd

access_token='2c0dac5d047d822efd0d4058af8b1973-b8b1e17acd17fb9bddd0b53adae9ae5f'

#2 - Define your signal function
def calculate_sma(data, period):
    return data['Close'].rolling(window=period).mean()

def signal_generator(df, short_period, long_period):
    df = df.copy()
    df['Short_SMA'] = calculate_sma(df, short_period)
    df['Long_SMA'] = calculate_sma(df, long_period)

    signal = []
    signal.append(0)
    
    for i in range(1, len(df)):
        if df['Short_SMA'].iloc[i] > df['Long_SMA'].iloc[i]:
            signal.append(1)  # Bullish Signal
        else:
            signal.append(2)  # Bearish Signal

    df['Signal'] = signal
    return df

#3 - Connect to the market and execute trades
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.GBP_CAD, Gran.H1)
    candles = collector.grab(n)
    return candles

def trading_job():
    candles = get_candles(30)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])

    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:], 9, 20)#

    # EXECUTING ORDERS
    accountID = "101-004-26226167-002" #your account ID here
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    #print(dfstream.iloc[:-1,:])
    print(f'TP Buy: {TPBuy} \n SL Buy: {SLBuy} \n TP Sell: {TPSell}\n SL Sell: {SLSell})
          
    #signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="GBP_CAD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="GBP_CAD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
        
#4 - Executing orders automatically with a scheduler
#trading_job()

scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
scheduler.start()

# Code with Create Orders

In [ ]:
import pandas as pd
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oanda_candles import Pair, Gran, CandleClient

access_token = '2c0dac5d047d822efd0d4058af8b1973-b8b1e17acd17fb9bddd0b53adae9ae5f'
accountID = "101-004-26226167-002"  # Your account ID here
client = API(access_token)

def calculate_sma(data, period):
    return data['Close'].rolling(window=period).mean()

def signal_generator(df, short_period, long_period):
    df = df.copy()
    df['Short_SMA'] = calculate_sma(df, short_period)
    df['Long_SMA'] = calculate_sma(df, long_period)

    signal = []
    signal.append(0)

    for i in range(1, len(df)):
        if df['Short_SMA'].iloc[i] > df['Long_SMA'].iloc[i]:
            signal.append(1)  # Bullish Signal
        else:
            signal.append(2)  # Bearish Signal

    df['Signal'] = signal
    return df

def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.GBP_CAD, Gran.H1)
    candles = collector.grab(n)
    return candles

# Define a function to get open trades
def get_open_trades(account_id):
    open_trades_request = trades.TradesList(accountID=account_id)
    open_trades_response = client.request(open_trades_request)
    print(open_trades_response['trades'])
    open_trades = open_trades_response['trades']
    return open_trades

    
def trading_job():
    candles = get_candles(30)
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', 'Low'])

    i = 0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i += 1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal_df = signal_generator(dfstream.iloc[:-1, :], 9, 20)
    signal = signal_df['Signal'].iloc[-1]

    previous_candleR = abs(dfstream['High'].iloc[-2] - dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o)) - 0.0005  # 5 pips as Stop Loss
    SLSell = float(str(candle.bid.o)) + 0.0005  # 5 pips as Stop Loss

    TPBuy = float(str(candle.bid.o)) + 0.0050  # 50 pips as Take Profit
    TPSell = float(str(candle.bid.o)) - 0.0050  # 50 pips as Take Profit

    print(f'TP Buy: {TPBuy}\nSL Buy: {SLBuy}\nTP Sell: {TPSell}\nSL Sell: {SLSell}')

    # Check if there is an open trade (you need to implement this logic)
    
    open_trades = get_open_trades(accountID)
    signal == 0
    if not open_trades:
        if signal == 1:
            mo = MarketOrderRequest(instrument="GBP_CAD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
            r = orders.OrderCreate(accountID, data=mo.data)
            rv = client.request(r)
            print(json.dumps(rv, indent=4))
        elif signal == 2:
            mo = MarketOrderRequest(instrument="GBP_CAD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
            r = orders.OrderCreate(accountID, data=mo.data)
            rv = client.request(r)
            print(json.dumps(rv, indent=4))
    else:
        print("Open Trades:")
        for trade in open_trades:
            print(f"Trade ID: {trade['id']}")
            print(f"Instrument: {trade['instrument']}")
            print(f"Initial Units: {trade['initialUnits']}")
            print(f"Current Units: {trade['currentUnits']}")
            print(f"Take Profit: {trade['takeProfitOrder']['price']}")
            print(f"Stop Loss: {trade['stopLossOrder']['price']}")
            print("----------------------")
        
#trading_job()
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'interval', hours=1)
#scheduler.start()


In [11]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2023-10-24 04:06:45'

# Code to execute only when there is a cross over

In [14]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import yfinance as yf
import pandas as pd
import datetime
import json

access_token = '2c0dac5d047d822efd0d4058af8b1973-b8b1e17acd17fb9bddd0b53adae9ae5f'
accountID = "101-004-26226167-002"  # Your account ID here
client = API(access_token)

def calculate_sma(data, period):
    return data['Close'].rolling(window=period).mean()

def signal_generator(df, short_period, long_period):
    df = df.copy()
    df['Short_SMA'] = calculate_sma(df, short_period)
    df['Long_SMA'] = calculate_sma(df, long_period)

    signal = []
    signal.append(0)

    for i in range(1, len(df)):
        if df['Short_SMA'].iloc[i] > df['Long_SMA'].iloc[i] and df['Short_SMA'].iloc[i-1] < df['Long_SMA'].iloc[i-1]:
            signal.append(1)  # Bullish Signal
        elif df['Short_SMA'].iloc[i] < df['Long_SMA'].iloc[i] and df['Short_SMA'].iloc[i-1] > df['Long_SMA'].iloc[i-1]:
            signal.append(2)  # Bearish Signal
        else:
            signal.append(0)  # No Signal

    df['Signal'] = signal
    return df

'''
def download_data(last_days):
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.datetime.today() - datetime.timedelta(days=last_days)).strftime('%Y-%m-%d')
    ticker_symbol = 'GBPCAD=X'  # Replace with the desired stock symbol
    data = yf.download(ticker_symbol, start=start_date, end=end_date, interval='1h')
    
    return data

d = signal_generator(download_data(4), 9, 20)
d.head(200)
'''

def get_candles(n):
    try:
        client = CandleClient(access_token, real=False)
        print("Connected to the API successfully!")
    except:
        print('Connection to API failed')
        sys.exit(1) 
        
    collector = client.get_collector(Pair.GBP_CAD, Gran.M15)
    candles = collector.grab(n)
    return candles

def trading_job():
    print(f'Job started at {datetime.datetime.today().strftime("%Y-%m-%d %H:%M:%S")}, execution pending...')
    candles = get_candles(30)
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', 'Low'])

    i = 0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i += 1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal_df = signal_generator(dfstream.iloc[:-1, :], 9, 20)
    signal = signal_df['Signal'].iloc[-1]

    previous_candleR = abs(dfstream['High'].iloc[-2] - dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle.bid.o)) - 0.0005  # 5 pips as Stop Loss
    SLSell = float(str(candle.bid.o)) + 0.0005  # 5 pips as Stop Loss

    TPBuy = float(str(candle.bid.o)) + 0.0050  # 50 pips as Take Profit
    TPSell = float(str(candle.bid.o)) - 0.0050  # 50 pips as Take Profit
    
    if signal == 1:
        print('buy market order executed')
        print(f'TP Buy: {TPBuy}\nSL Buy: {SLBuy}')
        mo = MarketOrderRequest(instrument="GBP_CAD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(json.dumps(rv, indent=4))
    elif signal == 2:
        print('sell market order executed')
        print(f'TP Sell: {TPSell}\nSL Sell: {SLSell}')
        mo = MarketOrderRequest(instrument="GBP_CAD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(json.dumps(rv, indent=4))

    print("Job executed successfully!\n===============================")
    
#trading_job()
scheduler = BlockingScheduler()
scheduler.add_job(trading_job, 'interval', minutes=15)
scheduler.start()

Job started at 2023-10-24 08:12:54, execution pending...
get_candles function called
Connected to the API successfully!
singal generator called
TP Buy: 1.68319
SL Buy: 1.6776900000000001
TP Sell: 1.6731900000000002
SL Sell: 1.67869
Job executed successfully!
Job started at 2023-10-24 08:27:54, execution pending...
get_candles function called
Connected to the API successfully!
singal generator called
TP Buy: 1.6833699999999998
SL Buy: 1.67787
TP Sell: 1.67337
SL Sell: 1.6788699999999999
Job executed successfully!
Job started at 2023-10-24 08:42:54, execution pending...
get_candles function called
Connected to the API successfully!
singal generator called
TP Buy: 1.6832399999999998
SL Buy: 1.67774
TP Sell: 1.67324
SL Sell: 1.67874
Job executed successfully!
Job started at 2023-10-24 08:57:54, execution pending...
get_candles function called
Connected to the API successfully!
singal generator called
TP Buy: 1.68248
SL Buy: 1.6769800000000001
TP Sell: 1.6724800000000002
SL Sell: 1.67798
Jo

KeyboardInterrupt: 